# Install Package

In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install surprise

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import dump

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Loading Data

In [ ]:
#anime = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/anime.csv")
#synopsis = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/anime_with_synopsis.csv")
#animelist = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/animelist.csv")
#rating_complete = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/rating_complete.csv")
#watching_status = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/animedataset/watching_status.csv")

In [ ]:
data = pd.read_csv("/content/drive/Shareddrives/DSL_2021_2_추천시스템/추천시스템 발표구현/Wide & Deep/anime_merge_sampling_100000.csv")

* we only need User_id, Item_id, Rating in Matrix Factorization

In [ ]:
cols = ['user_id','anime_id', 'rating', 'English name']
data = data[cols]

In [ ]:
#결측값 제거
data[data=='Unknown'] = np.nan
data.dropna(axis=0, inplace=True)

In [ ]:
data.shape

(80820, 4)

In [ ]:
data.head()

,user_id,anime_id,rating,English name
0,186051,18679,7,KILL la KILL
1,113524,12531,9,Kids on the Slope
2,175828,4224,9,Toradora!
3,252068,30276,9,One Punch Man
5,142479,23079,6,Glasslip


#Data Preprocessing

* Transform into Surprise dataset

In [ ]:
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1,10))

data = Dataset.load_from_df(data[['user_id','anime_id','rating']],
                            reader=reader)

# Training & Model 

Train-test split

In [ ]:
from surprise.model_selection import train_test_split

train,test = train_test_split(data, test_size=0.2, random_state=0)

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy

In [ ]:
bsl_options = {'method': 'sgd',
               'learning_rate': 0.001,
               }

In [ ]:
algo = SVD(n_factors=3, n_epochs=200, lr_all=0.001, reg_all=0.01, verbose=True) #n_factors 기존 5에서 3으로 수정
algo.fit(train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

# Prediction & Evaluation

##Prediction

1. test()를 사용해 모든 테스트 데이터에 대한 평점 예측

In [ ]:
prediction = algo.test(test)
print('prediction type: ', type(prediction),
     'size: ', len(prediction))
print()
print('prediction 결과값 5개 미리보기')
print(prediction[0])
print(prediction[1])
print(prediction[2])
print(prediction[3])
print(prediction[4])


prediction type:  <class 'list'> size:  16164

prediction 결과값 5개 미리보기
user: 204375     item: 19489      r_ui = 9.00   est = 7.86   {'was_impossible': False}
user: 179567     item: 2404       r_ui = 7.00   est = 7.73   {'was_impossible': False}
user: 207311     item: 392        r_ui = 7.00   est = 8.37   {'was_impossible': False}
user: 243635     item: 1571       r_ui = 7.00   est = 7.84   {'was_impossible': False}
user: 107844     item: 38295      r_ui = 7.00   est = 5.73   {'was_impossible': False}


In [ ]:
# user id, item id, 예측평점값들만 추출해서 하나의 튜플로 담겨있도록 하기
result = [(pred.uid, pred.iid, pred.est) for pred in prediction[:5]]
print(result)

[(204375, 19489, 7.854693877431593), (179567, 2404, 7.716512676754922), (207311, 392, 8.370836695469194), (243635, 1571, 7.8407185527690455), (107844, 38295, 5.757269306714182)]


2. predict()를 사용해 특정한 유저의 특정 아이템에 대한 평점 예측

In [ ]:
# user id, item id는 문자열로 되어있기 때문에 문자열로 넣어주어야 함!
uid = str(310448) #310448번 유저
iid = str(6178) #6178번 애니메이션(test_data 상 실제 평점 7.0)
# 변수 순서 지켜주어서 넣어주어야 함!
pred = algo.predict(uid, iid)
print(pred)

user: 310448     item: 6178       r_ui = None   est = 7.53   {'was_impossible': False}


## Evaluating

In [ ]:
prediction = algo.test(test)
print("RMSE of original model:{}".format(accuracy.rmse(prediction)))

RMSE: 1.5737
RMSE of original model:1.5737429451464156


정확도가 썩 좋아보이진 않는다.

# Hyperparameter Tuning

In [ ]:
from surprise.model_selection import GridSearchCV
#from scipy.stats import randint, uniform

# GridSearch 할 파라미터 사전적으로 정의
param_grid = {'n_factors':[1,2,3,4,5,6]}

# GridSearchCV는 cross_validate와는 달리 인자에 알고리즘 자체를 넣어준다!
grid = GridSearchCV(SVD, param_grid=param_grid,
                   measures=['rmse'], cv=3,
                    return_train_measures=True) # measure을 소문자로 해줘야함!

# GridSearchCV로 데이터 학습시키기
grid.fit(data)

# 최고의 score와 그 때의 파라미터 출력
print(grid.best_score['rmse'])
print(grid.best_params['rmse'])


1.5923545530646896
{'n_factors': 3}
